In [35]:
import os

import numpy as np
import pandas as pd
import plotly.express as px
import haven.db as db

from mirrorverse.utils import read_data_w_cache

os.environ['HAVEN_DATABASE'] = 'haven'
os.environ['AWS_PROFILE'] = 'admin'

In [8]:
train = read_data_w_cache(
    'select * from movement_model_features_m3_a2'
)
train.head()

,_decision,_choice,_selected,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,...,region,home_lat,home_lon,normed_home_lat,normed_home_lon,fork_length_cm,normalized_fork_length,cos_time,sin_time,tag_key
0,3,2,True,30.497318,14.465889,-2.608261,0.000000,0.000000,840c503ffffffff,840c503ffffffff,...,BC,57.0,-136.0,0.633333,-0.755556,81.0,0.777122,0.415415,0.909632,159012
1,5,9,False,39.001527,8.097725,1.131332,-0.701095,39.735291,840c503ffffffff,840c503ffffffff,...,BC,57.0,-136.0,0.633333,-0.755556,81.0,0.777122,0.415415,0.909632,159012
2,21,4,False,37.105423,23.591280,2.865548,0.686042,43.020737,840c5edffffffff,840c5edffffffff,...,BC,57.0,-136.0,0.633333,-0.755556,81.0,0.777122,-0.142315,0.989821,159012
3,4,18,False,9.453802,13.094096,-1.985158,0.674414,87.115089,840c503ffffffff,840c503ffffffff,...,BC,57.0,-136.0,0.633333,-0.755556,81.0,0.777122,0.415415,0.909632,159012
4,21,13,False,35.741338,27.489655,2.475744,-2.444469,42.879120,840c5edffffffff,840c5edffffffff,...,BC,57.0,-136.0,0.633333,-0.755556,81.0,0.777122,-0.142315,0.989821,159012


In [9]:
test = read_data_w_cache(
    'select * from movement_model_features_m3_a2_test'
)
test.head()

,_decision,_choice,_selected,mixed_layer_thickness,net_primary_production,water_heading,movement_heading,distance,origin_h3_index,next_h3_index,...,region,home_lat,home_lon,normed_home_lat,normed_home_lon,fork_length_cm,normalized_fork_length,cos_time,sin_time,tag_key
0,17,1,False,14.880235,5.140197,-2.849403,0.716714,85.654060,840ced1ffffffff,840ced1ffffffff,...,WA/OR,46.0,-125.0,0.511111,-0.694444,66.0,-1.350667,0.415415,-9.096320e-01,205406
1,38,5,False,10.490738,6.375038,-2.515185,2.915748,70.676826,840ced7ffffffff,840ced7ffffffff,...,WA/OR,46.0,-125.0,0.511111,-0.694444,66.0,-1.350667,0.841254,-5.406408e-01,205406
2,52,17,False,26.421632,1.916896,-1.537231,0.352988,75.999084,840ceb5ffffffff,840cccbffffffff,...,WA/OR,46.0,-125.0,0.511111,-0.694444,66.0,-1.350667,1.000000,-2.449294e-16,205406
3,42,19,False,13.405025,1.620341,-2.321418,1.568281,69.583122,840ced7ffffffff,840ced7ffffffff,...,WA/OR,46.0,-125.0,0.511111,-0.694444,66.0,-1.350667,0.841254,-5.406408e-01,205406
4,26,1,False,24.803674,4.895049,-2.911566,0.716714,85.654060,840ced1ffffffff,840ced1ffffffff,...,WA/OR,46.0,-125.0,0.511111,-0.694444,66.0,-1.350667,0.841254,-5.406408e-01,205406


In [10]:
train['_individual'].max()

90

In [11]:
test['_individual'].min()

0

In [12]:
test['_individual'] = test['_individual'] + 91

In [13]:
data = pd.concat([train, test])

In [17]:
data['binned_normed_distance'] = round(data['normed_distance'] * 10)/10
print(sorted(data['binned_normed_distance'].unique()))

[0.0, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]


In [22]:
data['binned_movement_heading'] = round(data['movement_heading'] / (np.pi/8)) * (np.pi/8)
print(sorted(data['binned_movement_heading'].unique()))

[-3.1415927, -2.7488937, -2.3561945, -1.9634955, -1.5707964, -1.1780972, -0.7853982, -0.3926991, 0.0, 0.3926991, 0.7853982, 1.1780972, 1.5707964, 1.9634955, 2.3561945, 2.7488937, 3.1415927]


In [30]:
df = data.groupby(['binned_normed_distance', 'binned_movement_heading']).size().reset_index()
df['size'] = np.log(df[0])
px.scatter(df, x='binned_normed_distance', y='binned_movement_heading', size='size')

In [33]:
data[['_train', '_individual']].drop_duplicates().groupby('_train').size()

_train
False    40
True     71
dtype: int64

In [37]:
db.write_data(
    data, 'movement_model_features_m3_a4', ['tag_key']
)